In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
dataframes=[]
stock_list=['INE423A01024','INE742F01042','INE437A01024','INE021A01026','INE238A01034','INE917I01010','INE918I01026','INE296A01024','INE397D01024','INE029A01011','INE216A01030','INE059A01026','INE522F01014','INE361B01024','INE089A01023','INE066A01021','INE047A01021','INE860A01027','INE040A01034','INE795G01014','INE158A01026','INE038A01020','INE030A01027','INE090A01021','INE095A01012','INE009A01021','INE154A01025','INE019A01038','INE237A01028','INE018A01030','INE214T01019','INE101A01026','INE585B01010','INE239A01024','INE733E01010','INE213A01029','INE752E01010','INE002A01018','INE123W01016','INE062A01020','INE044A01036','INE192A01025','INE155A01022','INE081A01020','INE467B01029','INE669C01036','INE280A01028','INE481G01011','INE628A01036','INE075A01022' ]

In [4]:
def n_day_regression(n,data):
  lr=LinearRegression()
  X=list(range(n))
  X=np.array(X)

  _varname_ = f'{n}_reg'
  data[_varname_]=np.nan

  for i in range(n-1,len(data)):
    y=[]
    for j in range(0,n):
      y.append(data["Close"][i-j])
    y=y[::-1]
    y=np.array(y)
    X=X.reshape(X.shape[0],1)
    y=y.reshape(y.shape[0],1)
    lr.fit(X,y)
    coef=lr.coef_[0][0]
    data.loc[i,_varname_]=coef

  return data

In [5]:
def local_maxima_minima(data,n=5):

  data["local_min"]=np.nan
  data["local_max"]=np.nan
  print(len(data))
  for i in range(n,len(data)-n):
    l = data["Close"][i-n:i+n]
    if data["Close"][i]==min(l):
      data.loc[i,"local_min"]=1
      data.loc[i,"local_max"]=0
    elif data["Close"][i]==max(l):

      data.loc[i,"local_max"]=1
      data.loc[i,"local_min"]=0
    else:
      data.loc[i,"local_max"]=0
      data.loc[i,"local_min"]=0

  return data

In [6]:
for i in stock_list:
    data = pd.read_csv(f"./data/{i}.csv")
    data['Normalised_value'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])
    data_tmp=data.copy()

    data['Normalised_value'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])
    
    data_tmp2=local_maxima_minima(data_tmp)
    local_minima_indices = np.where(data_tmp2['local_min'] > 0)[0]
    local_maxima_indices = np.where(data_tmp2['local_max'] > 0)[0]
    
    data_tmp2=n_day_regression(3,data_tmp2)
    data_tmp2=n_day_regression(5,data_tmp2)
    data_tmp2=n_day_regression(10,data_tmp2)
    data_tmp2=n_day_regression(20,data_tmp2)

    extremity_data=data_tmp2[(data_tmp2['local_min'] > 0) | (data_tmp2['local_max'] > 0)].reset_index(drop = True)
    extremity_data=extremity_data.dropna()

    extremity_data["target"]=[1 if x > 0 else 0 for x in extremity_data.local_max]
    extremity_data=extremity_data.drop(["Time","Open","Close","High","Low","OpenInterest","local_min","local_max"],axis=1).reset_index(drop=True)

    dataframes.append(extremity_data)


    



1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1239
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238
1238


In [9]:
merged_df_minima = pd.concat(dataframes_minima, ignore_index=True)
merged_df_maxima = pd.concat(dataframes_maxima, ignore_index=True)

In [10]:
merged_df_minima.to_csv('merged_minima_data.csv', index=False)
merged_df_maxima.to_csv('merged_maxima_data.csv', index=False)